In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
pd.read_csv('AAPL.csv')

,date,close,volume,open,high,low
0,2016/11/04,108.8400,30790930.0,108.5300,110.2500,108.1100
1,2016/11/03,109.8300,26538700.0,110.9800,111.4600,109.5500
2,2016/11/02,111.5900,28174980.0,111.4000,112.3500,111.2300
3,2016/11/01,111.4900,43403760.0,113.4600,113.7700,110.5300
4,2016/10/31,113.5400,26378910.0,113.6500,114.2300,113.2000
5,2016/10/28,113.7200,36792100.0,113.8700,115.2100,113.4500
6,2016/10/27,114.4800,31396130.0,115.3900,115.8600,114.1000
7,2016/10/26,115.5900,66028640.0,114.3100,115.7000,113.3100
8,2016/10/25,118.2500,46820600.0,117.9500,118.3600,117.3100
9,2016/10/24,117.6500,23492650.0,117.1000,117.7400,117.0000
